# WordNet

In [2]:
from collections import defaultdict

import networkx as nx
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import wordnet as wn

## Loading data

### Maslow's paper

In [3]:
maslow_df = pd.read_excel('data/aux/maslow.xlsx')
maslow_df.fillna('', inplace=True)

In [4]:
maslow_df.head()

,PHYSIOLOGICAL,SAFETY,LOVE,ESTEEM,SELF_ACTUALIZATION,SELF_TRANSCENDENCE
0,food,freedom,love,esteem,write,volunteering
1,water,philosophy,community,evaluation,poetry,help
2,salt,safety,parents,respect,history,morality
3,sugar,stable,family,capacity,philosophy,ethics
4,protein,sunniness,sex,achievement,culture,volunteering


### Other sources

In [5]:
others_dict = defaultdict(list)
with open('data/aux/maslow-others.txt') as f:
    c = 1
    for line in f:
        if c % 7 != 1 and c % 7 != 0:
            col, vals = line.split(':')
            vals = vals.strip()
            if vals:
                col = col.replace('-', '_').upper()
                vals = vals.split(', ')
                others_dict[col].extend(vals)
        c += 1

others_df = pd.DataFrame(
    dict([(k, pd.Series(v)) for k, v in others_dict.items()]),
    columns=['PHYSIOLOGICAL', 'SAFETY', 'LOVE', 'ESTEEM', 'SELF_ACTUALIZATION', 'SELF_TRANSCENDENCE']
)
others_df.fillna('', inplace=True)

In [6]:
others_df.head()

,PHYSIOLOGICAL,SAFETY,LOVE,ESTEEM,SELF_ACTUALIZATION,SELF_TRANSCENDENCE
0,air,assault,friends,wise,capable,
1,water,murder,family,decision-maker,education,
2,calories,homicide,community,status,university,
3,nutrients,chaos,divorce,confidence,college,
4,food,war,child,competent,creativity,


### Concatenating data

In [7]:
needs_df = pd.concat([maslow_df, others_df], ignore_index=True)

In [8]:
needs_df.head()

,PHYSIOLOGICAL,SAFETY,LOVE,ESTEEM,SELF_ACTUALIZATION,SELF_TRANSCENDENCE
0,food,freedom,love,esteem,write,volunteering
1,water,philosophy,community,evaluation,poetry,help
2,salt,safety,parents,respect,history,morality
3,sugar,stable,family,capacity,philosophy,ethics
4,protein,sunniness,sex,achievement,culture,volunteering


## Processing data

In [9]:
def all_hypernyms(ss):
    return ss.closure(lambda x: x.hypernyms())

In [10]:
# Returns a dictionary: {hypon_1: [hyper__1-1, ..., hyper__1-m_1], ..., hypon__n: [hyper__n-1, ..., hyper__n-m_n]}
def hyponyms_hypernyms(lemma1, lemma2):
    result = {}
    sss1 = wn.synsets(lemma1)
    sss2 = wn.synsets(lemma2)
    for ss1 in sss1:
        hypernyms1 = set(all_hypernyms(ss))
        intersect = hypernyms1.intersection(sss2)
        if intersect:
            result[ss1] = list(intersect)
    return result

In [11]:
for need in needs_df:
    print(need, needs_df[need].nunique())

PHYSIOLOGICAL 53
SAFETY 142
LOVE 71
ESTEEM 70
SELF_ACTUALIZATION 124
SELF_TRANSCENDENCE 6


### Physiological needs

In [24]:
needs_df.PHYSIOLOGICAL.sort_values().unique()

array(['', 'air', 'appetite', 'bread', 'breath', 'breathing', 'calcium',
       'calories', 'cigarette', 'clothing', 'comfort', 'defecate', 'drink',
       'eat', 'exercise', 'fat', 'fatigue', 'feed', 'food', 'health',
       'heat', 'hormones', 'hunger', 'hungry', 'hydrogen', 'movement',
       'nourishment', 'nutrients', 'oxygen', 'pleasure', 'protein', 'rest',
       'safe', 'salt', 'secure', 'sex', 'shelter', 'sleep', 'sleepiness',
       'smell', 'smoke', 'stroking', 'sugar', 'taste', 'temperature',
       'thirst', 'thirsty', 'tickling', 'urinate', 'vitamin', 'vitamins',
       'warmth', 'water'], dtype=object)

In [32]:
physiological_needs_removed = [
    '', 'bread', 'calcium', 'hormones', 'hydrogen', 'movement', 'salt', 'sugar', 'temperature', 'vitamins',
]

In [25]:
physiological_needs = [
    'air', 'appetite', 'breath', 'breathing', 'calories', 'cigarette', 'clothing', 'comfort', 'defecate', 'drink',
    'eat', 'exercise', 'fat', 'fatigue', 'feed', 'food', 'health', 'heat', 'hunger', 'hungry', 'nourishment',
    'nutrients', 'oxygen', 'pleasure', 'protein', 'rest', 'safe', 'secure', 'sex', 'shelter', 'sleep', 'sleepiness',
    'smell', 'smoke', 'stroking', 'taste', 'thirst', 'thirsty', 'tickling', 'urinate', 'vitamin', 'warmth', 'water',
]

In [26]:
physiological_all_synsets_list = [(need, ss) for need in physiological_needs for ss in wn.synsets(need)]

In [27]:
len(physiological_all_synsets_list)

262

In [15]:
physiological_synsets_list = [
    'air.n.01', # air
    'appetite.n.01', # appetite
    'bread.n.01', # bread
    'breath.n.01', 'breath.n.02', 'breath.n.03', # breath
    'breathing.n.01', 'breathe.v.01', 'rest.v.02', # breathing
    'calorie.n.01', 'calorie.n.02', # calories
    'cigarette.n.01', # cigarette
    'clothing.n.01', 'dress.v.02', 'clothe.v.03', # clothing
    'comfort.n.01', 'comfort.n.05',  # comfort
    'stool.v.04', # defecate
    'drink.n.01', 'drink.n.02', 'beverage.n.01', 'drink.v.01', 'drink.v.02', 'drink.v.05', # drink
    'eat.v.01', 'eat.v.02', # eat
    'exercise.n.01', 'exercise.v.03', 'exercise.v.04', # exercise
    'fat.n.01', 'fatness.n.01', 'fat.a.01', 'fatty.a.01', # fat
    'fatigue.n.01', 'fatigue.n.03', 'tire.v.01', 'tire.v.02', # fatigue
    'feed.n.01', 'feed.v.01', 'feed.v.02', 'feed.v.03', 'feed.v.07', # feed
    'food.n.01', 'food.n.02', # food
    'health.n.01', 'health.n.02', # health
    'heat.n.01', 'hotness.n.01', 'heat.n.03', 'heat.v.01', 'heat.v.02', 'inflame.v.04', 'heat.v.04', # heat
    'hunger.n.01', 'hunger.v.01', 'starve.v.01', # hunger
    'hungry.a.01', # hungry
    'nutriment.n.01', 'nourishment.n.02', # nourishment
    'food.n.01', # nutrients
    'oxygen.n.01', # oxygen
    'pleasure.n.01', 'joy.n.02', 'pleasure.n.04', 'pleasure.n.05', # pleasure
    'protein.n.01', # protein
    'rest.n.02', 'respite.n.04', 'rest.n.04', 'rest.v.01', 'rest.v.02', 'rest.v.03', 'rest.v.05', # rest
    'condom.n.01', 'safe.a.01', 'safe.s.02', #safe
    'secure.a.02', # secure
    'sexual_activity.n.01', 'sex.n.03', 'sex.n.04', 'arouse.v.07', # sex
    'shelter.n.01', 'shelter.n.02', 'protection.n.04', 'shelter.n.05', 'shelter.v.01', # shelter
    'sleep.n.01', 'sleep.n.02', 'sleep.n.03', 'sleep.v.01', 'sleep.v.02', # sleep
    'sleepiness.n.01', # sleepiness
    'smell.n.01', 'olfactory_property.n.01', 'smell.n.04', 'smell.n.05', 'smell.v.01', # smell
    'roll_of_tobacco.n.01', 'pot.n.09', 'smoke.n.07', 'smoke.v.01', # smoke
    'stroke.n.05', 'stroke.v.01', # stroking
    'taste.n.01', 'preference.n.01', 'taste.n.05', 'taste.v.02', 'taste.v.05', # taste
    'thirst.n.01', 'thirst.v.01', # thirst
    'thirsty.s.01', 'thirsty.a.02', # thirsty
    'tickle.n.02', 'tickle.v.01', 'thrill.v.02', 'tickle.v.03', 'tickling.s.01', # tickling
    'urinate.v.01', 'make.v.49', # urinate
    'vitamin.n.01', # vitamin
    'heat.n.03', 'warmheartedness.n.01', 'warmth.n.03', # warmth
    'water.n.01', 'urine.n.01', 'water.n.06', # water
]

In [28]:
for need, ss in physiological_all_synsets_list:
    print('*', need, '-', ss.name(), '-', ss.definition())

* air - air.n.01 - a mixture of gases (especially oxygen) required for breathing; the stuff that the wind consists of
* air - air.n.02 - the region above the ground
* air - air.n.03 - a distinctive but intangible quality surrounding a person or thing
* air - breeze.n.01 - a slight wind (usually refreshing)
* air - atmosphere.n.03 - the mass of air surrounding the Earth
* air - air.n.06 - once thought to be one of four elements composing the universe (Empedocles)
* air - tune.n.01 - a succession of notes forming a distinctive sequence
* air - air.n.08 - medium for radio and television broadcasting
* air - air_travel.n.01 - travel via aircraft
* air - air_out.v.01 - expose to fresh air
* air - air.v.02 - be broadcast
* air - air.v.03 - broadcast over the airwaves, as in radio or television
* air - publicize.v.01 - make public
* air - air.v.05 - expose to warm or heated air, so as to dry
* air - vent.v.02 - expose to cool or cold air so as to cool or freshen
* appetite - appetite.n.01 - a

* thirsty - thirsty.a.02 - feeling a need or desire to drink
* thirsty - athirst.s.01 - (usually followed by `for') extremely desirous
* thirsty - thirsty.s.04 - able to take in large quantities of moisture
* tickling - tickle.n.02 - the act of tickling
* tickling - tickle.v.01 - touch (a body part) lightly so as to excite the surface nerves and cause uneasiness, laughter, or spasmodic movements
* tickling - thrill.v.02 - feel sudden intense sensation or emotion
* tickling - tickle.v.03 - touch or stroke lightly
* tickling - tickling.s.01 - exciting by touching lightly so as to cause laughter or twitching movements
* urinate - urinate.v.01 - pass after the manner of urine
* urinate - make.v.49 - eliminate urine
* vitamin - vitamin.n.01 - any of a group of organic substances essential in small quantities to normal metabolism
* warmth - heat.n.03 - the sensation caused by heat energy
* warmth - warmheartedness.n.01 - a warmhearted feeling
* warmth - warmth.n.03 - the quality of having a 

In [23]:
for ss_str in physiological_synsets_list.copy():
    ss1 = wn.synset(ss_str)
    for lemma in ss1.lemmas():
        for antonym in lemma.antonyms():
            ss2 = antonym.synset()
            physiological_synsets_list.append(ss2.name())

In [24]:
physiological_synsets_list = sorted(set(physiological_synsets_list))

In [25]:
len(physiological_synsets_list)

121

In [33]:
physiological_aux_synsets_graph = nx.DiGraph()

for ss_str in physiological_synsets_list:
    physiological_aux_synsets_graph.add_node(ss_str)

for ss_str in physiological_aux_synsets_graph:
    ss = wn.synset(ss_str)
    for hyper in all_hypernyms(ss):
        if hyper in physiological_aux_synsets_graph:
            physiological_aux_synsets_graph.add_edge(hyper, ss)

In [35]:
physiological_aux_synsets_graph.number_of_nodes()

121

In [38]:
physiological_aux_synsets_graph.number_of_edges()

0

In [43]:
qqq=set()
for ss_str in physiological_synsets_list:
    ss = wn.synset(ss_str)
    qqq.add(ss)
    for hypo in ss.closure(lambda x: x.hyponyms()):
        qqq.add(hypo)

In [44]:
len(qqq)

4740

In [ ]:
Eliminar el paso de quitar los hyponyms

In [6]:
PHYSIOLOGICAL_synsets = {}
for x in needs_df.PHYSIOLOGICAL.unique():
    PHYSIOLOGICAL_synsets[x] = wn.synsets(x)

SyntaxError: unexpected EOF while parsing (<ipython-input-6-3cb35bf3bf65>, line 1)

In [127]:
for ss in wn.synsets('thirsty'):
    print(ss, ss.definition())

Synset('thirsty.s.01') needing moisture
Synset('thirsty.a.02') feeling a need or desire to drink
Synset('athirst.s.01') (usually followed by `for') extremely desirous
Synset('thirsty.s.04') able to take in large quantities of moisture


In [29]:
physiological_synsets_list

['air.n.01',
 'appetite.n.01',
 'arouse.v.07',
 'be_active.v.01',
 'be_full.v.01',
 'breath.n.01',
 'breathe.v.01',
 'breathing.n.01',
 'calcium.n.01',
 'calorie.n.01',
 'carbohydrate.n.01',
 'clothe.v.03',
 'clothing.n.01',
 'comfort.n.01',
 'comfort.n.05',
 'condom.n.01',
 'cool.v.01',
 'dangerous.a.01',
 'discomfort.n.01',
 'dress.v.02',
 'drink.n.01',
 'drink.n.02',
 'drink.v.01',
 'drink.v.02',
 'drink.v.05',
 'ease.n.02',
 'eat.v.02',
 'exercise.n.01',
 'exercise.v.03',
 'exercise.v.04',
 'fat.a.01',
 'fat.n.01',
 'fatigue.n.01',
 'fatness.n.01',
 'fatty.a.01',
 'feed.v.01',
 'feed.v.02',
 'feed.v.03',
 'food.n.01',
 'food.n.02',
 'health.n.01',
 'health.n.02',
 'heat.n.01',
 'heat.v.01',
 'heat.v.02',
 'heating_system.n.01',
 'hunger.n.01',
 'hunger.v.01',
 'hungry.a.01',
 'illness.n.01',
 'insecure.a.02',
 'joy.n.02',
 'leanness.n.02',
 'make.v.49',
 'nonfat.a.01',
 'nourishment.n.02',
 'olfactory_property.n.01',
 'oxygen.n.01',
 'pain.n.02',
 'pleasure.n.01',
 'pleasure.n.04',

In [7]:
wn.synsets('adfassdasd')

[]

In [8]:
wn.synsets('educated')

[Synset('educate.v.01'),
 Synset('train.v.01'),
 Synset('educate.v.03'),
 Synset('educated.a.01'),
 Synset('educated.s.02')]

In [9]:
wn.synsets('education')

[Synset('education.n.01'),
 Synset('education.n.02'),
 Synset('education.n.03'),
 Synset('education.n.04'),
 Synset('education.n.05'),
 Synset('department_of_education.n.01')]

In [14]:
wn.synsets('hungry')

[Synset('hungry.a.01'), Synset('athirst.s.01')]

In [15]:
ss=wn.synsets('hungry')[0]
ss.causes()

[]

In [16]:
ss=wn.synsets('hungry')[1]
ss.causes()

[]

In [17]:
wn.synsets('Person')

[Synset('person.n.01'), Synset('person.n.02'), Synset('person.n.03')]

In [18]:
ss=wn.synsets('Person')[0]
ss.hypernyms()

[Synset('causal_agent.n.01'), Synset('organism.n.01')]

In [19]:
ss.hypernyms()[0].definition()

'any entity that produces an effect or is responsible for events or results'

In [21]:
ss.hypernyms()[0].hyponyms()

[Synset('agent.n.01'),
 Synset('agent.n.03'),
 Synset('catalyst.n.02'),
 Synset('cause_of_death.n.01'),
 Synset('danger.n.03'),
 Synset('destiny.n.02'),
 Synset('deus_ex_machina.n.01'),
 Synset('engine.n.02'),
 Synset('first_cause.n.01'),
 Synset('nature.n.02'),
 Synset('operator.n.02'),
 Synset('person.n.01'),
 Synset('power.n.05'),
 Synset('supernatural.n.01'),
 Synset('theurgy.n.01'),
 Synset('vital_principle.n.01')]

In [26]:
ss=wn.synsets('hunger')[0]
list(ss.closure(lambda x: x.hypernyms()))

[Synset('drive.n.09'),
 Synset('physical_condition.n.01'),
 Synset('condition.n.01'),
 Synset('state.n.02'),
 Synset('attribute.n.02'),
 Synset('abstraction.n.06'),
 Synset('entity.n.01')]

In [27]:
list(wn.synset('physical_condition.n.01').closure(lambda x: x.hyponyms()))

[Synset('abnormality.n.01'),
 Synset('acathexia.n.01'),
 Synset('addiction.n.01'),
 Synset('agalactia.n.01'),
 Synset('amyxia.n.01'),
 Synset('analgesia.n.01'),
 Synset('anesthesia.n.01'),
 Synset('anestrus.n.01'),
 Synset('angiotelectasia.n.01'),
 Synset('anhidrosis.n.01'),
 Synset('arousal.n.02'),
 Synset('asphyxia.n.01'),
 Synset('bloodiness.n.01'),
 Synset('cellularity.n.01'),
 Synset('cold_sweat.n.01'),
 Synset('cryptobiosis.n.01'),
 Synset('dilatation.n.01'),
 Synset('disorder.n.01'),
 Synset('drive.n.09'),
 Synset('elastosis.n.01'),
 Synset('emmetropia.n.01'),
 Synset('estrus.n.01'),
 Synset('false_pregnancy.n.01'),
 Synset('fertility.n.02'),
 Synset('flatulence.n.01'),
 Synset('flexure.n.01'),
 Synset('good_health.n.01'),
 Synset('hypercapnia.n.01'),
 Synset('hyperpigmentation.n.01'),
 Synset('hyperthermia.n.01'),
 Synset('hypocapnia.n.01'),
 Synset('hypopigmentation.n.01'),
 Synset('hypothermia.n.01'),
 Synset('incompatibility.n.02'),
 Synset('muscularity.n.01'),
 Synset('myas

In [28]:
list(wn.synset('causal_agent.n.01').closure(lambda x: x.hyponyms()))

[Synset('agent.n.01'),
 Synset('agent.n.03'),
 Synset('catalyst.n.02'),
 Synset('cause_of_death.n.01'),
 Synset('danger.n.03'),
 Synset('destiny.n.02'),
 Synset('deus_ex_machina.n.01'),
 Synset('engine.n.02'),
 Synset('first_cause.n.01'),
 Synset('nature.n.02'),
 Synset('operator.n.02'),
 Synset('person.n.01'),
 Synset('power.n.05'),
 Synset('supernatural.n.01'),
 Synset('theurgy.n.01'),
 Synset('vital_principle.n.01'),
 Synset('biological_agent.n.01'),
 Synset('eliminator.n.01'),
 Synset('impairer.n.01'),
 Synset('infectious_agent.n.01'),
 Synset('lethal_agent.n.01'),
 Synset('motor.n.02'),
 Synset('mutagen.n.01'),
 Synset('quickener.n.01'),
 Synset('relaxer.n.01'),
 Synset('repressor.n.01'),
 Synset('retardant.n.01'),
 Synset('satisfier.n.01'),
 Synset('shortener.n.01'),
 Synset('stressor.n.01'),
 Synset('teratogen.n.01'),
 Synset('vasoconstrictor.n.01'),
 Synset('vector.n.03'),
 Synset('weakener.n.01'),
 Synset('antacid.n.01'),
 Synset('antiflatulent.n.01'),
 Synset('antifungal.n.01

In [35]:
is_hyponym('thirst', 'physical_condition')

True

In [36]:
is_hyponym('physical_condition', 'thirst')

False

In [37]:
is_hypernym('physical_condition', 'thirst')

True

In [49]:
wn.synsets('sculpt')

[Synset('sculpt.v.01'), Synset('sculpt.v.02')]

In [45]:
wn.synsets('sculpt')[0].definition()

'create by shaping stone or wood or any other hard material'

In [46]:
wn.synsets('sculpt')[1].definition()

'shape (a material like stone or wood) by whittling away at it'

In [47]:
list(wn.synsets('sculpt')[0].closure(lambda x: x.hypernyms()))

[Synset('shape.v.02'),
 Synset('create_from_raw_material.v.01'),
 Synset('make.v.03')]

In [48]:
list(wn.synsets('sculpt')[1].closure(lambda x: x.hypernyms()))

[Synset('carve.v.01'),
 Synset('cut.v.01'),
 Synset('shape.v.02'),
 Synset('separate.v.02'),
 Synset('create_from_raw_material.v.01'),
 Synset('move.v.02'),
 Synset('make.v.03')]

In [51]:
wn.synsets('paint')

[Synset('paint.n.01'),
 Synset('key.n.09'),
 Synset('rouge.n.01'),
 Synset('paint.v.01'),
 Synset('paint.v.02'),
 Synset('paint.v.03'),
 Synset('paint.v.04')]

In [55]:
wn.synsets('paint')[3].definition()

'make a painting'

In [56]:
wn.synsets('paint')[4].definition()

'apply paint to; coat with paint'

In [57]:
wn.synsets('paint')[5].definition()

'make a painting of'

In [58]:
wn.synsets('paint')[6].definition()

'apply a liquid to; e.g., paint the gutters with linseed oil'

In [59]:
list(wn.synsets('paint')[3].closure(lambda x: x.hypernyms()))

[Synset('create.v.03'), Synset('act.v.01')]

In [61]:
list(wn.synsets('paint')[5].closure(lambda x: x.hypernyms()))

[Synset('represent.v.09'), Synset('re-create.v.01'), Synset('make.v.03')]

In [125]:
basic = [
    wn.synset('food.n.01'),
    wn.synset('food.n.02'),
    wn.synset('water.n.01'),
    wn.synset('water.n.06'),
    wn.synset('protein.n.01'),
    wn.synset('calcium.n.01'),
    wn.synset('sexual_activity.n.01'),
    wn.synset('sex.n.03'),
    wn.synset('arouse.v.07'),
    wn.synset('pleasure.n.01'),
    wn.synset('pleasure.n.05'),
    wn.synset('comfort.n.01'),
    wn.synset('comfort.n.02'),
    wn.synset('consolation.n.02'),
    wn.synset('ease.n.02'),
    wn.synset('comfort.n.05'),
    wn.synset('comfort.v.02'),
    wn.synset('drink.v.01'),
    wn.synset('drink.v.02'),
    wn.synset('roll_of_tobacco.n.01'),
    wn.synset('pot.n.09'),
    wn.synset('smoke.n.07'),
    wn.synset('smoke.v.01'),
    wn.synset('feed.v.01'),
    wn.synset('feed.v.02'),
    wn.synset('feed.v.03'),
    wn.synset('eat.v.01'),
    wn.synset('eat.v.02'),
    wn.synset('sleep.n.01'),
    wn.synset('sleep.n.02'),
    wn.synset('sleep.n.03'),
    wn.synset('sleep.v.01'),
    wn.synset('sleep.v.02'),
    wn.synset('air.n.01'),
    wn.synset('clothing.n.01'),
    wn.synset('dress.v.02'),
    wn.synset('clothe.v.03'),
    wn.synset('shelter.n.01'),
    wn.synset('shelter.n.02'),
    wn.synset('protection.n.04'),
    wn.synset('shelter.n.05'),
    wn.synset('shelter.v.01'),
    wn.synset('heat.n.03'),
    wn.synset('warmth.n.03'),
    wn.synset('rest.n.02'),
    wn.synset('respite.n.04'),
    wn.synset('rest.n.04'),
    wn.synset('rest.v.02'),
    wn.synset('lie.v.06'),
    wn.synset('rest.v.05'),
    wn.synset('perch.v.01'),
    wn.synset('pillow.v.01'),
    wn.synset('appetite.n.01')
    wn.synset('hunger.n.01'),
    wn.synset('hunger.v.01'),
    wn.synset('crave.v.01'),
    wn.synset('starve.v.01'),
    wn.synset('sleepiness.n.01'),
    
    
    
    
    wn.synset('food.n.03'),
    wn.synset('body_of_water.n.01'),
    wn.synset('water.n.03'),
    wn.synset('water_system.n.02'),
    wn.synset('urine.n.01'),
]

In [130]:
term = 'fatigue'
sss = wn.synsets(term)
for ss in sss:
    for ss in [ss]+ss.part_holonyms()+ss.part_meronyms():
        inters = set(ss.closure(lambda x: x.hypernyms(), depth=-1)).intersection(basic)
        if inters:
            print(ss, inters)

In [131]:
for ss in sss:
    print(ss, ss.definition())

Synset('fatigue.n.01') temporary loss of strength and energy resulting from hard physical or mental work
Synset('fatigue.n.02') used of materials (especially metals) in a weakened state caused by long stress
Synset('fatigue.n.03') (always used with a modifier) boredom resulting from overexposure to something
Synset('fatigue_duty.n.01') labor of a nonmilitary kind done by soldiers (cleaning or digging or draining or so on)
Synset('tire.v.01') lose interest or become bored with something or somebody
Synset('tire.v.02') exhaust or get tired through overuse or great strain or stress


In [ ]:
Synset('fatigue.n.01') temporary loss of strength and energy resulting from hard physical or mental work
Synset('fatigue.n.02') used of materials (especially metals) in a weakened state caused by long stress
Synset('fatigue.n.03') (always used with a modifier) boredom resulting from overexposure to something
Synset('fatigue_duty.n.01') labor of a nonmilitary kind done by soldiers (cleaning or digging or draining or so on)
Synset('tire.v.01') lose interest or become bored with something or somebody
Synset('tire.v.02') exhaust or get tired through overuse or great strain or stress

In [82]:
# terms = [
#     'food', 'water', 'salt', 'sugar', 'protein', 'fat', 'calcium', 'oxygen', 'hydrogen', 'vitamins', 'sex',
#     'pleasure', 'taste', 'smell', 'tickling', 'stroking', 'comfort', 'drink', 'smoke', 'cigarette', 'feed',
#     'eat', 'bread', 'sleep', 'air', 'clothing', 'shelter', 'warmth', 'rest', 'nourishment', 'appetite',
#     'hunger', 'sleepiness', 'fatigue', 'hungry', 'thirst'
# ]
# for term in terms:
#     print('*'*80)
#     print('TERM:', term)
#     sss = wn.synsets(term)
#     print(sss)
#     print('='*80)
#     for ss in sss:
#         print('\t', 'SYNSET:', ss)
#         print('\t', 'Definition:', ss.definition())
#         print()
#         print('\t', 'HYPERNYMS')
#         print('\t', list(ss.closure(lambda x: x.hypernyms(), depth=-1)))
# #         print('\t', ss.hypernyms())
#         print()
#         print('\t', 'HIPONYMS')
# #         print('\t', list(ss.closure(lambda x: x.hyponyms(), depth=-1)))
#         print('\t', ss.hyponyms())
#         print()
#         print('\t', 'HOLONYMS')
#         print('\t', ss.part_holonyms())
#         print()
#         print('\t', 'MERONYMS')
#         print('\t', ss.part_meronyms())
#         print('-'*80)